In [1]:
from tqdm.contrib.concurrent import process_map

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.decode import print_breakdown, search

# TODO: handle non-US cars
df_nhtsa = await search(
  min_model_year=2019,  # TODO: increase range later
  include_openpilot=True,
  include_police=True,
  skip_missing_asbuilt=True,
)

# pre-load asbuilt
process_map(AsBuiltData.from_vin, df_nhtsa['VIN'].unique(), desc='Loading AsBuilt Data', chunksize=100)

print()
print_breakdown(df_nhtsa, include_model_year=False)

Loaded 36766 VINs (filter_comment=None, include_openpilot=True, skipped=162, missing_asbuilt=0)


Loading AsBuilt Data:   0%|          | 0/32344 [00:00<?, ?it/s]


Model
                      2
Aviator            1107
Bronco              997
Bronco Sport       1500
Continental          23
Corsair             986
E-Transit           157
Ecosport            322
Edge               2450
Escape             3133
Expedition          725
Expedition MAX      629
Explorer           2971
F-150              6541
F-150 Lightning     385
F-250              1500
F-350              1034
F-450               196
F-550                 1
Fiesta              147
Flex                137
Fusion              908
GT                    3
MKC                  55
MKT                   7
MKZ                 100
Maverick            994
Mustang             871
Mustang Mach-E      839
Nautilus           1169
Navigator           303
Navigator L         230
Ranger              533
Taurus               40
Transit            1009
Transit Connect     340
dtype: int64


In [2]:
df = df_nhtsa[['VIN', 'Make', 'Model', 'ModelYear']].copy()
df['CarName'] = df['Make'] + ' ' + df['Model'] + ' ' + df['ModelYear'].astype(str)
df.drop(columns=['Make', 'Model', 'ModelYear'], inplace=True)
df.head()

,VIN,CarName
0,1FTER4EH5KLA32672,FORD Ranger 2019
1,1FMCU9G60MUA32828,FORD Escape 2021
2,1FTVW1EL0NWG13995,FORD F-150 Lightning 2022
3,5LM5J7WC0PGL05389,LINCOLN Aviator 2023
5,1FTEW1CB8KFC33387,FORD F-150 2019


In [3]:
import pandas as pd

from panda.python.uds import DATA_IDENTIFIER_TYPE
from notebooks.ford.ecu import FordEcu

ecu_map = {
  'ABS': FordEcu.AntiLockBrakeSystem,
  'APIM': FordEcu.AccessoryProtocolInterfaceModule,
  'IPMA': FordEcu.ImageProcessingModuleA,
  'PSCM': FordEcu.PowerSteeringControlModule,
}

def get_ecu_platform_code(abd: AsBuiltData, ecu: FordEcu) -> str | None:
  fw = abd.get_identifier(ecu, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER)
  if fw is None:
    return None
  prefix, core, _ = fw.split('-')
  return f'{core}-{prefix}'

def get_platform_codes(row) -> pd.Series:
  abd = AsBuiltData.from_vin(row['VIN'])
  values: dict[str, str | None] = {}
  for ecu_name, ecu in ecu_map.items():
    values[ecu_name] = get_ecu_platform_code(abd, ecu)
  return pd.Series(values)

df_fw = df.join(df.apply(get_platform_codes, axis=1))
df_fw.head()

,VIN,CarName,ABS,APIM,IPMA,PSCM
0,1FTER4EH5KLA32672,FORD Ranger 2019,14C036-KB3C,14G374-1U5T,14F397-KB3T,14D003-JR3C
1,1FMCU9G60MUA32828,FORD Escape 2021,2D053-LX6C,14G374-1U5T,14F397-LJ6T,14D003-LX6C
2,1FTVW1EL0NWG13995,FORD F-150 Lightning 2022,2D053-PL38,14G676-PU5T,14H102-ML3T,14D003-NL38
3,5LM5J7WC0PGL05389,LINCOLN Aviator 2023,2D053-L1MC,14G374-1U5T,14F397-LB5T,14D003-M1MC
5,1FTEW1CB8KFC33387,FORD F-150 2019,2D053-KL34,14G374-1U5T,14F397-KL3T,14D003-KL3V


In [4]:
from notebooks.utils.union import merge_sets


def group_by(df: pd.DataFrame, by: list[str]) -> None:
  car_groups = list()
  for group in df.groupby(by, dropna=False):
    cars = set(group[1]['CarName'].unique())
    car_groups.append(cars)

  return merge_sets(car_groups)

In [5]:
from notebooks.ford.platforms import find_openpilot_platform

for ecus in (
  # ('ABS', 'PSCM'), - combines escape and bronco sport
  # ('ABS', 'APIM'), - combines escape and bronco sport
  # ('IPMA', 'PSCM'),
  # ('IPMA', 'APIM'),
  # ('PSCM', 'IPMA', 'APIM'), - combines escape and bronco sport
  # ('ABS', 'PSCM', 'APIM'),  - combines escape and bronco sport
  # ('ABS', 'IPMA', 'APIM'),
  ('ABS', 'IPMA'),
  ('ABS', 'IPMA', 'PSCM'),  # splits out: 2019 Mustang (from 2020-23), 2024 Bronco (from 2021-23)
  ('ABS', 'IPMA', 'PSCM', 'APIM'),  # splits out: 2019 EcoSport (from 2020-22), 2024 Escape/Corsair (from 2023)
):
  car_groups = group_by(df_fw, list(ecus))
  print(', '.join(ecus))
  print(f'Found {len(car_groups)} distinct car groups')
  for group in car_groups:
    # platforms = {find_openpilot_platform(car) for car in group}
    # print(f'{len(platforms)} platforms: {", ".join(map(str, platforms)):<33}; {", ".join(group)}')
    print('\t' + ', '.join(group))
  print()

ABS, IPMA
Found 50 distinct car groups
	FORD Transit 2019
	FORD Escape 2019
	FORD Ranger 2023, FORD Ranger 2022, FORD Ranger 2019, FORD Ranger 2021, FORD Ranger 2020
	FORD Fiesta 2019
	FORD Taurus 2019, LINCOLN MKT 2019, FORD Flex 2019
	LINCOLN Continental 2019, LINCOLN Continental 2020
	FORD Ecosport 2022, FORD Ecosport 2020, FORD Ecosport 2021, FORD Ecosport 2019
	FORD Explorer 2019
	FORD GT 2020
	LINCOLN Navigator 2019, FORD Expedition 2019, FORD Expedition MAX 2019, LINCOLN Navigator L 2019, LINCOLN  2019
	LINCOLN Nautilus 2019
	FORD Edge 2019
	FORD F-350 2019, FORD F-250 2019, FORD F-450 2019
	LINCOLN MKZ 2020, LINCOLN MKZ 2019, FORD Fusion 2019, FORD Fusion 2020
	LINCOLN MKC 2019
	LINCOLN Navigator 2020, FORD Expedition MAX 2020, LINCOLN Navigator L 2020, FORD Expedition 2020
	FORD F-150 2019, FORD F-150 2020
	FORD Mustang 2019, FORD Mustang 2022, FORD Mustang 2021, FORD Mustang 2023, FORD Mustang 2020
	FORD Transit Connect 2019, FORD Transit Connect 2020, FORD Transit Connect 20

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/camer/AppData/Local/Programs/Microsoft VS Code


In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

df_reduced_ecus = df_fw.groupby(by=['ABS', 'IPMA'])[['CarName']].agg(set)
df_reduced_ecus['CarInfoPlatforms'] = df_reduced_ecus['CarName'].apply(lambda x: {find_openpilot_platform(car_name) for car_name in x})
df_reduced_ecus

CarName  \
ABS         IPMA                                                                                                                                                                                                                              
14C036-FK41 14F397-CK4T                                                                                                                                                                                                 {FORD Transit 2019}   
14C036-GV61 14F397-F1FT                                                                                                                                                                                                  {FORD Escape 2019}   
14C036-KB3C 14F397-KB3T                                                                                                                          {FORD Ranger 2023, FORD Ranger 2022, FORD Ranger 2019, FORD Ranger 2021, FORD Ranger 2020}   
2D053-FG13  14G019-DA5T                                                                                                                                                                                {FORD Taurus 2019, LINCOLN MKT 2019}   
            14G019-EG1T                                                                                                                                                                                {FORD Taurus 2019, LINCOLN MKT 2019}   
2D053-G3GC  14G019-GD9T                                                                                                                                                                {LINCOLN Continental 2019, LINCOLN Continental 2020}   
2D053-HB53  14G019-GB5T                                                                                                                                                                                                {FORD Explorer 2019}   
2D053-JL14  14F397-JL1T                                                                                                                  {LINCOLN Navigator 2019, FORD Expedition MAX 2019, FORD Expedition 2019, LINCOLN Navigator L 2019}   
2D053-K2GC  14F397-KA1T                                                                                                                                                                                             {LINCOLN Nautilus 2019}   
            14F397-KT4T                                                                                                                                                                                                    {FORD Edge 2019}   
2D053-KC3C  14G019-HC3T                                                                                                                                                                 {FORD F-350 2019, FORD F-250 2019, FORD F-450 2019}   
2D053-KG9C  14G019-HS7T                                                                                                                                            {LINCOLN MKZ 2020, LINCOLN MKZ 2019, FORD Fusion 2019, FORD Fusion 2020}   
2D053-KJ7C  14G019-GJ7T                                                                                                                                                                                                  {LINCOLN MKC 2019}   
            14G019-KJ7T                                                                                                                                                                                                  {LINCOLN MKC 2019}   
2D053-KL14  14F397-LL1T                                                                                                                  {LINCOLN Navigator 2020, FORD Expedition MAX 2020, LINCOLN Navigator L 2020, FORD Expedition 2020}   
2D053-KL34  14F397-JL3T                                                                                                                                              